# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 3-month treasury bond

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=53

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==53]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = df3['interest_rate']/100
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_ttm53', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_ttm53/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-08-07 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31066/1242795185.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31066/1242795185.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31066/1242795185.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"


...

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]         [,4]
sigmas -1.340278958 1.4134121 -1.710336247  2.246070816
       -1.307052188 1.3983723 -1.690504792  2.236294037
       -1.274894026 1.3835086 -1.670768046  2.226548807
       -1.243737871 1.3688200 -1.651125415  2.216834402
       -1.213523165 1.3543057 -1.631576312  2.207150083
       -1.184194675 1.3399647 -1.612120157  2.197495095
       -1.155701894 1.3257961 -1.592756376  2.187868663
       -1.127998511 1.3117991 -1.573484404  2.178269998
       -1.101041963 1.2979726 -1.554303682  2.168698289
       -1.074793036 1.2843159 -1.535213658  2.159152710
       -1.049215530 1.2708280 -1.516213790  2.149632413
       -1.024275949 1.2575082 -1.497303543  2.140136532
       -0.999943247 1.2443554 -1.478482389  2.130664179
       -0.976188587 1.2313690 -1.459749811  2.121214446
       -0.952985143 1.2185479 -1.441105299  2.111786406
       -0.930307911 1.2058915 -1.422548353  2.102379107
       -0.9081

        0.428728967 0.6754065  0.223410743  0.650719702
        0.434472842 0.6774162  0.229050829  0.636422383
        0.440183913 0.6794748  0.234560837  0.622104238
        0.445862553 0.6815811  0.239940689  0.607766393
        0.451509127 0.6837340  0.245190325  0.593409974
        0.457123997 0.6859323  0.250309698  0.579036099
        0.462707516 0.6881748  0.255298777  0.564645886
        0.468260033 0.6904604  0.260157548  0.550240450
        0.473781889 0.6927880  0.264886011  0.535820903
        0.479273422 0.6951562  0.269484182  0.521388355
        0.484734962 0.6975641  0.273952092  0.506943913
        0.490166836 0.7000104  0.278289789  0.492488683
        0.495569364 0.7024939  0.282497334  0.478023767
        0.500942861 0.7050136  0.286574807  0.463550269
        0.506287638 0.7075682  0.290522299  0.449069287
        0.511604001 0.7101567  0.294339922  0.434581923
        0.516892249 0.7127778  0.298027798  0.420089274
        0.522152678 0.7154305  0.301586070  0.40

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31066/1242795185.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31066/1242795185.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_31066/1242795185.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]          [,4]
sigmas -0.858334730 1.1655289 -1.532195691  2.7587669138
       -0.838935421 1.1527683 -1.508255508  2.7353307127
       -0.819905295 1.1402137 -1.484544604  2.7120357618
       -0.801230562 1.1278631 -1.461061642  2.6888833828
       -0.782898191 1.1157144 -1.437805238  2.6658748020
       -0.764895855 1.1037654 -1.414773969  2.6430111502
       -0.747211880 1.0920140 -1.391966371  2.6202934611
       -0.729835201 1.0804583 -1.369380940  2.5977226706
       -0.712755321 1.0690960 -1.347016134  2.5752996164
       -0.695962272 1.0579252 -1.324870377  2.5530250364
       -0.679446577 1.0469438 -1.302942058  2.5308995692
       -0.663199226 1.0361497 -1.281229533  2.5089237527
       -0.647211635 1.0255410 -1.259731128  2.4870980241
       -0.631475631 1.0151155 -1.238445142  2.4654227194
       -0.615983417 1.0048715 -1.217369844  2.4438980734
       -0.600727555 0.9948067 -1.196503482  2.42252421

        0.471811179 0.6713166  0.221354105  0.3525690888
        0.476977818 0.6731771  0.224168682  0.3367263826
        0.482117900 0.6750602  0.226848617  0.3208627777
        0.487231697 0.6769649  0.229393732  0.3049787650
        0.492319476 0.6788899  0.231803863  0.2890748595
        0.497381500 0.6808342  0.234078857  0.2731516003
        0.502418029 0.6827965  0.236218573  0.2572095503
        0.507429319 0.6847757  0.238222881  0.2412492957
        0.512415621 0.6867707  0.240091663  0.2252714458
        0.517377183 0.6887803  0.241824814  0.2092766330
        0.522314249 0.6908034  0.243422243  0.1932655121
        0.527227061 0.6928388  0.244883872  0.1772387609
        0.532115855 0.6948854  0.246209635  0.1611970790
        0.536980864 0.6969420  0.247399482  0.1451411884
        0.541822320 0.6990075  0.248453375  0.1290718330
        0.546640449 0.7010808  0.249371294  0.1129897785
        0.551435475 0.7031606  0.250153231  0.0968958124
        0.556207618 0.7052458  

In [2]:
print(error_i)

[0, 1, 4, 5, 6, 7, 8, 9, 11, 12, 14, 15, 17, 18]
